<a href="https://colab.research.google.com/github/heroza/Keel/blob/main/pima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%load_ext autoreload
%autoreload 2

In [7]:
import myfunction as fu
import numpy as np
import pandas as pd
from typing_extensions import Counter
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [8]:
df = pd.read_csv('pima.dat', header=None)


In [9]:
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,positive
1,1,85,66,29,0,26.6,0.351,31,negative
2,8,183,64,0,0,23.3,0.672,32,positive
3,1,89,66,23,94,28.1,0.167,21,negative
4,0,137,40,35,168,43.1,2.288,33,positive


In [10]:
num_features=8

le = LabelEncoder()
X = df.loc[:, df.columns[0:num_features]].to_numpy()

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

y = df.loc[:, df.columns[-1]].to_numpy()
y = le.fit_transform(y)
# y_train = y_train.reshape(-1)
print(X.shape)
print(Counter(y))

(768, 8)
Counter({0: 500, 1: 268})


In [11]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
thresholds = [0.02, 0.05, 0.1, 0.3, 0.5]
ks = [3,5,7,9]
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

for k in ks:
  for threshold in thresholds:
    scores = []
    for i in range(3):
      for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        # print(f"Fold {fold+1}")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        sia_model = fu.train_siamese_dense(X_train, y_train, X_test, y_test, num_classes=2, num_features=num_features, epochs_siamese=100, batch_size=16)

        # Initialize the SVM classifier
        clf = svm.SVC()

        X_train_resampled, y_train_resampled = fu.sia_smote_balance_dense(X_train, y_train, sia_model, threshold, maxdist_from_base=0.5, k = k)

        # Train the classifier on the training set
        clf.fit(X_train_resampled, y_train_resampled)

        # Predict the labels of the test set
        y_pred = clf.predict(X_test)

        # Calculate the F1 score
        score = fu.calculate_score(y_test, y_pred)
        scores.append(score)

        # Print the F1 score
        print(f"Score in i:{i}, fold:{fold}:", score)

    print(f'k={k} and threshold={threshold}')
    print('printing score')
    fu.print_score(scores)

creating pairs
Counter({1.0: 426, 0.0: 426})
Counter({1.0: 106, 0.0: 106})
init base network
training siamese network...
evaluate
7/7 [==============================] - 0s 2ms/step - loss: 0.2738
predict train
27/27 [==============================] - 0s 1ms/step
compute train acc
predict val
7/7 [==============================] - 0s 2ms/step
compute val acc
Loss = 0.2737758159637451, Train Accuracy = 0.8028169014084507 Test Accuracy = 0.5754716981132075
6/6 [==============================] - 0s 2ms/step
Score in i:0, fold:0: [0.7337662337662337, 0.7737037037037038, 0.7620634755325443, 0.7050359712230215, 0.5764705882352941, 0.9074074074074074, 0.64, 0.7737037037037037]
creating pairs
Counter({1.0: 426, 0.0: 426})
Counter({1.0: 106, 0.0: 106})
init base network
training siamese network...
evaluate
7/7 [==============================] - 0s 2ms/step - loss: 0.2803
predict train
27/27 [==============================] - 0s 1ms/step
compute train acc
predict val
7/7 [========================

In [12]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
thresholds = [0.02, 0.05, 0.1, 0.3, 0.5]
ks = [3,5,7,9]
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

for k in ks:
  for threshold in thresholds:
    scores = []
    for i in range(3):
      for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        # print(f"Fold {fold+1}")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        sia_model = fu.train_siamese_dense(X_train, y_train, X_test, y_test, num_classes=2, num_features=num_features, epochs_siamese=100, batch_size=16)

        # Initialize the SVM classifier
        clf = svm.SVC()

        X_train_resampled, y_train_resampled = fu.sia_smote_then_smote_dense(X_train, y_train, sia_model, threshold, maxdist_from_base=0.5, k = k)

        # Train the classifier on the training set
        clf.fit(X_train_resampled, y_train_resampled)

        # Predict the labels of the test set
        y_pred = clf.predict(X_test)

        # Calculate the F1 score
        score = fu.calculate_score(y_test, y_pred)
        scores.append(score)

        # Print the F1 score
        print(f"Score in i:{i}, fold:{fold}:", score)

    print(f'k={k} and threshold={threshold}')
    print('printing score')
    fu.print_score(scores)

creating pairs
Counter({1.0: 426, 0.0: 426})
Counter({1.0: 106, 0.0: 106})
init base network
training siamese network...
evaluate
7/7 [==============================] - 0s 2ms/step - loss: 0.3072
predict train
27/27 [==============================] - 0s 1ms/step
compute train acc
predict val
7/7 [==============================] - 0s 2ms/step
compute val acc
Loss = 0.3072112202644348, Train Accuracy = 0.8133802816901409 Test Accuracy = 0.5849056603773585
6/6 [==============================] - 0s 2ms/step
Score in i:0, fold:0: [0.7402597402597403, 0.7488888888888889, 0.7483314773547882, 0.6774193548387097, 0.6, 0.7777777777777778, 0.72, 0.7488888888888888]
creating pairs
Counter({1.0: 426, 0.0: 426})
Counter({1.0: 106, 0.0: 106})
init base network
training siamese network...
evaluate
7/7 [==============================] - 0s 2ms/step - loss: 0.2540
predict train
27/27 [==============================] - 0s 1ms/step
compute train acc
predict val
7/7 [==============================] - 0s 2m